In [81]:
%load_ext autoreload
%autoreload 2

from dask_image.ndfilters import median_filter, threshold_local, percentile_filter
from dask_image.ndfilters import convolve, maximum_filter
from pyfires.PYF_WindowStats import get_mea_std_window
from satpy.modifiers.angles import _get_sun_angles
from cython.parallel import prange

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob
from pyfires.PYF_detection import stage1_tests, run_basic_night_detection

from pyfires.PYF_detection import stage1_tests, run_basic_night_detection, _make_kern
import pyfires.PYF_Consts as PYFc
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import xarray as xr
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from satpy import Scene
from glob import glob
import os
bdict = {'vi1_band': 'B03',
         'vi2_band': 'B06',
         'mir_band': 'B07',
         'lwi_band': 'B13'}


curfile = 'D:/sat_data/ahi_main/in/0840/HS_H09_20231029_0840_B07_FLDK_R20_S0110.DAT'
tod = 'D:/sat_data/ahi_main/out/'
pos = curfile.find('B07')
dtstr = curfile[pos-14:pos-1]
ifiles_l15 = glob(f'{os.path.dirname(curfile)}/*{dtstr}*.DAT')
scn = Scene(ifiles_l15, reader='ahi_hsd')
scn.load(['B03', 'B13'])
scn = scn.resample(scn.coarsest_area(), resampler='native')
#scn.save_datasets(base_dir=tod, enhance=False, dtype=np.float32)
data_dict = initial_load(ifiles_l15,  # List of files to load
                         'ahi_hsd',   # The reader to use, in this case the AHI HSD reader
                         bdict,       # The bands to load
                         mir_bt_thresh=270, # The threshold for the MIR band, pixels cooler than this are excluded.
                         lw_bt_thresh=260)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


cannot convert float NaN to integer


In [82]:
data_dict['PFP'] = stage1_tests(data_dict['MIR__BT'],
                                data_dict['BTD'],
                                data_dict['VI1_DIFF'],
                                data_dict['SZA'],
                                data_dict['LSM'],
                                ksizes=[5, 7, 9],
                                do_lsm_mask=True)

In [85]:
from pyspectral.solar import SolarIrradianceSpectrum as SIS
from pyspectral.rsr_reader import RelativeSpectralResponse

scn = Scene(ifiles_l15, reader='ahi_hsd')
scn2 = Scene(ifiles_l15, reader='ahi_hsd')
mir_band = 'B07'
lwi_band = 'B13'

blist = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06']

blist = blist + [mir_band]
scn2.load(['B13'])
scn.load(blist + ['B07'], calibration='radiance')
scn = scn.resample(scn.coarsest_area(), resampler='native')

sensor = scn[mir_band].attrs['sensor']
platform = scn[mir_band].attrs['platform_name']
srf = RelativeSpectralResponse(platform, sensor)

cur_rsr = srf.rsr[mir_band]
wvl_mir = cur_rsr['det-1']['central_wavelength']

exp_rad = calc_rad_fromtb(scn2[lwi_band].data, wvl_mir)

out_rad = {}
for band_name in blist + ['B07']:
    cur_rsr = srf.rsr[band_name]
    irrad = SIS().inband_solarirradiance(cur_rsr)
    out_rad[band_name] = irrad
    
scn[mir_band].data = scn[mir_band].data - exp_rad

scn[mir_band].data = scn[mir_band].where(scn[mir_band].data > 0.001)

saa, sza_data = _get_sun_angles(scn[mir_band])
sza_data = xr.where(sza_data > 85, 85, sza_data)

for band_name in blist:
    print(out_rad[mir_band] / out_rad[band_name])
    scn[band_name].data = (np.pi * scn[band_name].data / out_rad[band_name]) / np.cos(np.deg2rad(sza_data))
    #scn[band_name].data = scn[band_name].data * (out_rad[mir_band] / out_rad[band_name])


#    scn[band_name].data = scn[band_name].data / scn[mir_band].data
#    scn[band_name ].data = scn[band_name].where(np.isfinite(scn[mir_band].data))

#scn.save_datasets(base_dir=tod, enhance=False, dtype=np.float32)

0.005226888907252563
0.005519401138531416
0.0063395004736882905
0.010535507086029615
0.04184438839166822
0.13831809322871033
1.0


In [86]:
bob = (scn['B07'] - scn['B04'] > 0)
#save_output(scn, bob, 'bob', tod)

In [88]:
save_output(scn, bob * data_dict['PFP'], 'bob2', tod)